In [39]:
# load data
from cot import Collection
coll = Collection("open_book_qa", load_pregenerated_cots=False)
coll = coll.select(split="test", number_samples=1)
coll

Reusing dataset open_book_qa_dataset (/Users/robertpraas/.cache/huggingface/datasets/open_book_qa_dataset/thoughtsource/1.0.0/b1163057b19c0b0be4f0f1104103cb49d35de01177baec5250adb5246116272f)


Loading open_book_qa...


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4957 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

| Name         | Train   | Valid   |   Test |
|--------------|---------|---------|--------|
| open_book_qa | -       | -       |      1 |

Not loaded: ['aqua', 'asdiv', 'commonsense_qa', 'entailment_bank', 'gsm8k', 'mawps', 'med_qa', 'med_qa_open', 'medmc_qa', '_init_', 'mmlu_clinical_knowledge', 'mmlu_college_biology', 'mmlu_college_medicine', 'mmlu_medical_genetics', 'mmlu_professional_medicine', '_init_', 'mmlu_anatomy', 'pubmed_qa', 'qed', 'strategy_qa', 'svamp', 'worldtree']

In [40]:
# load local model

from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [41]:
"""Create a sequential langchain consisting of one for reasoning generation and the 
other for answer extraction. Inspired by https://js.langchain.com/docs/modules/chains/sequential_chain"""
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain


In [42]:
"""Reasoning chain"""

llm = local_llm

template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

{cot_trigger}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger"], template=template)
cot_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="cot")

In [43]:
"""answer extraction chain"""

extraction_template = """{instruction}

Question: {question}
Answer_choices: {answer_choices}

Cot: {cot_trigger}{cot}
{answer_extraction}
"""

prompt_template = PromptTemplate(input_variables=["instruction","question","answer_choices","cot_trigger","cot","answer_extraction"], template=extraction_template)
answer_chain = LLMChain(llm=llm, prompt=prompt_template,output_key="predicted_answer")

In [44]:
"""combination of generation and extraction chain"""

from langchain.chains import SequentialChain
overall_chain = SequentialChain(chains=[cot_chain, answer_chain],
                                input_variables=["instruction","question","answer_choices","cot_trigger","answer_extraction"],
                                output_variables=["cot", "predicted_answer"],
                                verbose=True)

In [45]:
"""Create an input dictionary to feed the chain"""
input_dict = {'input_dict':
              {
                  'chain': overall_chain,
                  "instruction": "Be faithful and a little hopeful",
                  "cot_trigger": "Answer: Let's think step by step.",
                  "answer_extraction": "Therefore, among A through D, the answer is",
                  'model': "flan-T5-small",
                  'temperature': 0,
                  'max_tokens': 800
              }
              }

"""generate_extract_flexible is called in the dataloade.py file where the collection resides.
 From there a function in the generate.py file is loaded. Next to this function you could use the functions
 {generate/extract/metareason}_flexible"""    
coll.generate_extract_flexible(input_dict=input_dict)

Generating open_book_qa...


  0%|          | 0/1 [00:00<?, ?ex/s]



> Entering new SequentialChain chain...

> Finished chain.
{'id': 'open_book_qa_test_432', 'ref_id': '', 'question': 'How many times would someone change the page of a calendar in a year?', 'type': 'multiplechoice', 'choices': ['13', '12', '15', '14'], 'context': '', 'cot': ['one year is equal to 365 days'], 'answer': ['12'], 'generated_cot': [{'id': 'b2e8943b-f4a7-4d6c-a9d6-68d132d6f38b', 'fragments_version': None, 'instruction': 'Be faithful and a little hopeful', 'cot_trigger': "Answer: Let's think step by step.", 'cot_trigger_template': '', 'prompt_text': '', 'cot': 'C', 'answers': [{'id': 'd9ebc5e8-cc5d-41af-9ce6-8411a3744604', 'answer_extraction': 'Therefore, among A through D, the answer is', 'answer_extraction_template': '', 'answer_extraction_text': '', 'answer': 'A', 'answer_from_choices': '', 'correct_answer': None}], 'author': '', 'date': '2023/05/12 11:59:34', 'api_service': '', 'model': "{'name': 'flan-T5-small', 'temperature': 0, 'max_tokens': 800}", 'comment': 'genera

In [46]:
# Evaluate the answer given
coll.evaluate(title='custom_model_title')

  0%|          | 0/1 [00:00<?, ?ex/s]

{'open_book_qa': {'test': {'accuracy': {'flan-T5-small': {'custom_model_title': 0.0}}}}}